In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
import ktrain
from ktrain import text
from sklearn.metrics import ConfusionMatrixDisplay
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer 
import re
import string
from nltk.tokenize import RegexpTokenizer

In [26]:
dataset1=pd.read_csv("all_data.csv")

In [27]:
dataset1.head()

,Unnamed: 0,text,sentiment
0,0,﻿حقا\n,pos
1,1,صح جدا\n,pos
2,2,﻿اه و النعمة \n,pos
3,3,كلامك جميل ورائع\n,pos
4,4,﻿و خير الكلام ما قل و دل\n,pos


In [28]:
dataset1pos = dataset1[dataset1.sentiment == 'pos']
dataset1pos.shape

(93144, 3)

In [29]:
dataset1neg = dataset1[dataset1.sentiment == 'neg']
dataset1neg.shape

(63263, 3)

In [30]:
dataset1pos = dataset1pos.sample(frac=0.67,random_state=200)

dataset1pos.shape

(62406, 3)

In [31]:
dataset1 = pd.concat([dataset1pos, dataset1neg], axis=0)
dataset1.head()

,Unnamed: 0,text,sentiment
46539,44548,حاتم خيمي ( رئيس نادي الوحدة ) الهلال في وادي ...,pos
24590,22599,استفيد بخصومات حتي % و % 🎇 🎇 و استفيد أيضا بخص...,pos
60381,58390,مايخذ من وقتك دقائق ، البقيات الصالحات 💚 💭 سبح...,pos
46920,44929,حياة زوجيه سعـيده بين الشريكين ❤ 👩 ‍ اطلب مجمو...,pos
75256,73265,الناس صايره تتباهى بعدم الاخلاق 🙂\n,pos


In [32]:
dataset1= dataset1.drop('Unnamed: 0',axis=1)
dataset1.head()

,text,sentiment
46539,حاتم خيمي ( رئيس نادي الوحدة ) الهلال في وادي ...,pos
24590,استفيد بخصومات حتي % و % 🎇 🎇 و استفيد أيضا بخص...,pos
60381,مايخذ من وقتك دقائق ، البقيات الصالحات 💚 💭 سبح...,pos
46920,حياة زوجيه سعـيده بين الشريكين ❤ 👩 ‍ اطلب مجمو...,pos
75256,الناس صايره تتباهى بعدم الاخلاق 🙂\n,pos


In [34]:
dataset1["sentiment"].value_counts()

neg    63263
pos    62406
Name: sentiment, dtype: int64

In [35]:
dataset1.isnull().any(axis=0)

text         False
sentiment    False
dtype: bool

In [36]:
cols = ['sentiment','text']
positive = pd.read_csv('Arabic_tweets_positive.tsv',sep='\t', error_bad_lines = False ,header=None, names=cols)
positive.head()

,sentiment,text
0,pos,هه خلي فضولش ف جيبش 🌚
1,pos,موجود ماسافرت 😊
2,pos,: مسابقة #43 #متابعي_شامخ فقط# 🔘السحب على200💰ر...
3,pos,اللهم امين 🌺
4,pos,يعني اذا ما احبج احب منو؟ 💘


In [37]:
cols = ['sentiment','text']
negative = pd.read_csv('Arabic_tweets_negative.tsv',sep='\t', error_bad_lines = False ,header=None, names=cols)
negative.head()

,sentiment,text
0,neg,انمي Family Guy الموسم الحلقة السادسة مترجمة H...
1,neg,صدااع من الغباء لي ساعه افكر وش اقول بس غبائها...
2,neg,قتيلا وجريحا… حصيلة تفرقة المتظاهرين في كربلاء...
3,neg,اللهم من أصلح شأنهم وجمع كلمتهم ووحد صفهم ياقا...
4,neg,كوميديا.. الذهبية تذهب بالخطأ لصاحبة المركز ال...


In [38]:
dataset2 = pd.concat([positive, negative], axis=0)
dataset2.head()

,sentiment,text
0,pos,هه خلي فضولش ف جيبش 🌚
1,pos,موجود ماسافرت 😊
2,pos,: مسابقة #43 #متابعي_شامخ فقط# 🔘السحب على200💰ر...
3,pos,اللهم امين 🌺
4,pos,يعني اذا ما احبج احب منو؟ 💘


In [39]:
final_data = pd.concat([dataset2, dataset2], axis=0)
final_data.head()

,sentiment,text
0,pos,هه خلي فضولش ف جيبش 🌚
1,pos,موجود ماسافرت 😊
2,pos,: مسابقة #43 #متابعي_شامخ فقط# 🔘السحب على200💰ر...
3,pos,اللهم امين 🌺
4,pos,يعني اذا ما احبج احب منو؟ 💘


In [40]:
final_data.sentiment.value_counts()

pos    152188
neg    144468
Name: sentiment, dtype: int64

In [41]:
final_data.shape

(296656, 2)

In [42]:
for letter in '#.][!XR':
    final_data['text'] = final_data['text'].astype(str).str.replace(letter,'')

In [43]:
arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations

def remove_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)

In [44]:
def remove_non_arabic(text):
    return ' '.join(re.sub(u"[^\u0621-\u063A\u0640-\u0652 ]", " ", str(text),  flags=re.UNICODE).split())

In [45]:
def normalize_arabic(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    return text

In [46]:
def remove_repeating_char(text):
    return re.sub(r'(.)\1+', r'\1', text)

In [47]:
def remove_emoji(text):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

In [48]:
def processPost(tweet): 

    #Replace @username with empty string
    tweet = re.sub('@[^\s]+', ' ', tweet)
    
    #Convert www.* or https?://* to " "
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ',tweet)
    
    #Replace #word with word
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)

    # remove punctuations
    tweet= remove_punctuations(tweet)
    
    # normalize the tweet
    tweet= normalize_arabic(tweet)
    
    # remove repeated letters
    #tweet=remove_repeating_char(tweet)


    # remove emoji
    tweet=remove_emoji(tweet)
    
    return tweet

In [49]:
final_data["text"] = final_data['text'].apply(lambda x: processPost(x))

In [50]:
final_data["text"] = final_data['text'].apply(remove_non_arabic)

In [51]:
final_data.head()

,sentiment,text
0,pos,هه خلي فضولش ف جيبش
1,pos,موجود ماسافرت
2,pos,مسابقه متابعيشامخ فقط السحب علي ريال الشروط مت...
3,pos,اللهم امين
4,pos,يعني اذا ما احبج احب منو


In [64]:
final_data.shape

(296656, 2)

In [53]:
final_data["text"] = final_data['text'].apply(lambda x:remove_repeating_char(x))

In [31]:
#tokenizer = RegexpTokenizer(r'\w+')
#final_data["tweet_text"] = final_data["tweet_text"].apply(tokenizer.tokenize)

In [73]:
final_data["text"].sample(20)

12373                                اصبحنا واصبح الملك له
18809    سؤال الاسبوع اذكروا الايات والسور التي ذكر فيه...
49710                                         صباح السعاده
51753                         مشوار اروح الجامعه عشان كلاس
67067    ليس مهما ان تكون الاول بعض الاحيان تاخر لترتاح...
69959                                        اله صورتك ايل
59695                                             الحمد له
56913    تعال صحبني واصبحك بالخير يالي صباح الكون مع طل...
44418                                        امين صح لسانك
42679    امور عليك فعلها لتشعر بالرضا التام عن نفسك مقا...
21767     هذا الظاهر خامس او سادس ستيل لكواي من بدا الدوري
3114     لجادين فقط الرجال لمن يعاني من سرعهالقذف تاخير...
63572    سحب علي مبلغ مالي كاش ريتويت لهذه التغريده ريت...
40402    ياسيدتي كنت اهم امراه في تاريخي قبل رحيل العام...
58246    فيديو كلمه السيدعبدالملكبدرالدينالحوثي خلال فع...
40553                                            لاسف يعني
60819                                       واحد مح د بل

In [76]:
msk = np.random.rand(len(final_data)) < 0.8
df_train = final_data[msk]
df_test = final_data[~msk]
df_train.shape

(237075, 2)

In [78]:
df_test.shape

(59581, 2)

In [79]:
df_test.sentiment.value_counts()

pos    30723
neg    28858
Name: sentiment, dtype: int64

In [80]:
df_train.sentiment.value_counts()

pos    121465
neg    115610
Name: sentiment, dtype: int64

In [75]:
#X = final_data['tweet_text'].values
#y=final_data['label'].values
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=2)

In [81]:
# set hyperparameters
maxlen = 64
batch_size = 16
lr = 2e-5
epochs = 3

In [82]:
MODEL_NAME = 'aubmindlab/bert-base-arabertv01'
t = text.Transformer(MODEL_NAME, maxlen=maxlen)

In [83]:
trn = t.preprocess_train(df_train.text.values, df_train.sentiment.values)
tst = t.preprocess_test(df_test.text.values, df_test.sentiment.values)

preprocessing train...
language: ar
train sequence lengths:
	mean : 11
	95percentile : 22
	99percentile : 25


Is Multi-Label? False
preprocessing test...
language: ar
test sequence lengths:
	mean : 11
	95percentile : 23
	99percentile : 25


In [85]:
model = t.get_classifier()
learner = ktrain.get_learner(model, train_data=trn, val_data=tst, batch_size=batch_size)

In [86]:
history = learner.fit_onecycle(lr, epochs)



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/3
14818/14818 [==============================] - 32810s 2s/step - loss: 0.3663 - accuracy: 0.8295 - val_loss: 0.2881 - val_accuracy: 0.8781
Epoch 2/3
14818/14818 [==============================] - 32284s 2s/step - loss: 0.2355 - accuracy: 0.8994 - val_loss: 0.2032 - val_accuracy: 0.9213
Epoch 3/3
14818/14818 [==============================] - 32843s 2s/step - loss: 0.1166 - accuracy: 0.9540 - val_loss: 0.1503 - val_accuracy: 0.9487


In [87]:
learner.validate(val_data=tst)

              precision    recall  f1-score   support

           0       0.94      0.95      0.95     28858
           1       0.95      0.95      0.95     30723

    accuracy                           0.95     59581
   macro avg       0.95      0.95      0.95     59581
weighted avg       0.95      0.95      0.95     59581



array([[27454,  1404],
       [ 1651, 29072]])

In [88]:
p = ktrain.get_predictor(learner.model, t)

In [90]:
p.predict("الهلال افضل فريق كورة")

'pos'

In [91]:
ktrain.load_predictor

<function ktrain.core.load_predictor(fpath, batch_size=32, custom_objects=None)>

In [92]:
p.save('ar-bert-model-datasetmarge')